# Librerias

In [1]:
import pandas as pd
import requests
import json 
import folium 
import os
from datetime import datetime, timedelta

# USGS
## API Documentation - Earthquake Catalog
### URL
# Geophysics Institutions Throughout the World
- https://www.usgs.gov/programs/earthquake-hazards/science/geophysics-institutions-throughout-world
- https://earthquake.usgs.gov/fdsnws/event/1/[METHOD[?PARAMETERS]]
 - metodo: query
 - Parámetros:
   - Ubicación: Las peticiones que utilicen tanto rectángulo como círculo devolverán la intersección, que puede estar vacía, utilízala con precaución.
      * Conterminous U.S. [24.6, 50] Latitude. [-125, -65] Longitude.

https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=1995-05-04&minlatitude=24.6&maxlatitude=50&minlongitude=-125&maxlongitude=-65&limit=19999

In [2]:
def sismo(url):
    # Establecer el tiempo de inicio desde hoy menos 20000 eventos
    endtime = datetime.today().strftime('%Y-%m-%dT%H:%M:%S')
    starttime = (datetime.today() - timedelta(days=20000)).strftime('%Y-%m-%dT%H:%M:%S')

    # Establecer los demás parámetros de solicitud
    params = {"format": "geojson", "starttime": starttime, "endtime": endtime, "minmagnitude": 2.5,
            "minlatitude": 24.6, "maxlatitude": 50, "minlongitude": -125, "maxlongitude": -65, "limit": 20000}    
    data = requests.get(url, params = params)
    data = json.loads(data.text)
    return data

url = r"https://earthquake.usgs.gov/fdsnws/event/1/query?"
dic_sismo = sismo(url)
count=0
for i in (dic_sismo["features"]):
    if i["properties"]['cdi']:
        count += 1
# Convertir los datos a un dataframe
df_sismo = pd.json_normalize(dic_sismo['features'])

In [3]:
#folder_path = "../DASHBOARD/CSV_ORIGINAL/"
#file_name = "raw_USA.csv"
#file_path = os.path.join(folder_path, file_name)
#
#df_sismo.to_csv(file_path, index=False)

In [4]:
df_sismo.head()

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,us6000kcxl,2.50,"12 km SE of Lincolnville, Kansas",1684438229827,1684466842499,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,2.0,...,",dyfi,origin,phase-data,",37.0,0.724000,0.45,95.0,mb_lg,earthquake,"M 2.5 - 12 km SE of Lincolnville, Kansas",Point,"[-96.8691, 38.4053, 5]"
1,Feature,nc73889641,3.72,"3km NE of Prattville, CA",1684425353710,1684460134151,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,48.0,...,",dyfi,focal-mechanism,moment-tensor,nearby-cit...",47.0,0.023960,0.17,35.0,mw,earthquake,"M 3.7 - 3km NE of Prattville, CA",Point,"[-121.1346667, 40.2251667, 8.4]"
2,Feature,nc73889591,2.85,"2km ENE of Prattville, CA",1684417575740,1684443257491,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,6.0,...,",dyfi,focal-mechanism,nearby-cities,origin,pha...",41.0,0.033800,0.23,35.0,md,earthquake,"M 2.9 - 2km ENE of Prattville, CA",Point,"[-121.1318333, 40.2155, 7.22]"
3,Feature,tx2023jqyx,2.60,"20 km WSW of Coyanosa, Texas",1684398894840,1684428432193,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,...,",origin,phase-data,",15.0,0.018328,0.20,62.0,ml,earthquake,"M 2.6 - 20 km WSW of Coyanosa, Texas",Point,"[-103.2609303, 31.1701984, 2.121118164]"
4,Feature,nc73889446,2.57,"2km SW of Hollister, CA",1684385013420,1684457230947,NaN,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,9.0,...,",dyfi,focal-mechanism,nearby-cities,origin,pha...",74.0,0.017670,0.09,77.0,md,earthquake,"M 2.6 - 2km SW of Hollister, CA",Point,"[-121.4096667, 36.8341667, 8.66]"


Borro las columnas que no necesito

In [5]:
df_sismo.columns

Index(['type', 'id', 'properties.mag', 'properties.place', 'properties.time',
       'properties.updated', 'properties.tz', 'properties.url',
       'properties.detail', 'properties.felt', 'properties.cdi',
       'properties.mmi', 'properties.alert', 'properties.status',
       'properties.tsunami', 'properties.sig', 'properties.net',
       'properties.code', 'properties.ids', 'properties.sources',
       'properties.types', 'properties.nst', 'properties.dmin',
       'properties.rms', 'properties.gap', 'properties.magType',
       'properties.type', 'properties.title', 'geometry.type',
       'geometry.coordinates'],
      dtype='object')

In [6]:
df_sismo.drop(columns=['type','properties.alert','properties.tz','properties.code','properties.url','properties.detail','properties.net'
                       ,'properties.status','properties.types','properties.sources','properties.magType','properties.type','geometry.type',
                       'properties.title'], axis=1, inplace=True)

Cambio el formato de la columna properties.time

In [7]:
df_sismo["properties.time"] = pd.to_datetime(df_sismo["properties.time"], unit="ms") 
df_sismo["properties.updated"] = pd.to_datetime(df_sismo["properties.updated"], unit="ms")

In [8]:
df_sismo['estado'] = df_sismo['properties.place'].str.split(',').str[1].str.strip()

df_sismo['properties.place'] = df_sismo['properties.place'].str.split(',', expand=True)[0]


In [9]:
df_sismo.head(5)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,estado
0,us6000kcxl,2.50,12 km SE of Lincolnville,2023-05-18 19:30:29.827,2023-05-19 03:27:22.499,2.0,4.1,NaN,0,97,",us6000kcxl,",37.0,0.724000,0.45,95.0,"[-96.8691, 38.4053, 5]",Kansas
1,nc73889641,3.72,3km NE of Prattville,2023-05-18 15:55:53.710,2023-05-19 01:35:34.151,48.0,3.8,3.728,0,231,",ew1684425360,nc73889641,us6000kcuy,nn00859666,",47.0,0.023960,0.17,35.0,"[-121.1346667, 40.2251667, 8.4]",CA
2,nc73889591,2.85,2km ENE of Prattville,2023-05-18 13:46:15.740,2023-05-18 20:54:17.491,6.0,2.5,NaN,0,126,",nc73889591,nn00859663,us6000kcte,",41.0,0.033800,0.23,35.0,"[-121.1318333, 40.2155, 7.22]",CA
3,tx2023jqyx,2.60,20 km WSW of Coyanosa,2023-05-18 08:34:54.840,2023-05-18 16:47:12.193,NaN,NaN,NaN,0,104,",tx2023jqyx,",15.0,0.018328,0.20,62.0,"[-103.2609303, 31.1701984, 2.121118164]",Texas
4,nc73889446,2.57,2km SW of Hollister,2023-05-18 04:43:33.420,2023-05-19 00:47:10.947,9.0,3.4,NaN,0,105,",nc73889446,",74.0,0.017670,0.09,77.0,"[-121.4096667, 36.8341667, 8.66]",CA


In [10]:
#Divido la columna geometry coordinates
df_sismo[['Longitud', 'Latitud', 'Profundidad']] = df_sismo['geometry.coordinates'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))

In [11]:
df_sismo.head(1)

,id,properties.mag,properties.place,properties.time,properties.updated,properties.felt,properties.cdi,properties.mmi,properties.tsunami,properties.sig,properties.ids,properties.nst,properties.dmin,properties.rms,properties.gap,geometry.coordinates,estado,Longitud,Latitud,Profundidad
0,us6000kcxl,2.5,12 km SE of Lincolnville,2023-05-18 19:30:29.827,2023-05-19 03:27:22.499,2.0,4.1,NaN,0,97,",us6000kcxl,",37.0,0.724,0.45,95.0,"[-96.8691, 38.4053, 5]",Kansas,-96.8691,38.4053,5


In [12]:
df_sismo.drop("geometry.coordinates", axis=1, inplace=True) 

In [13]:
df_sismo.columns

Index(['id', 'properties.mag', 'properties.place', 'properties.time',
       'properties.updated', 'properties.felt', 'properties.cdi',
       'properties.mmi', 'properties.tsunami', 'properties.sig',
       'properties.ids', 'properties.nst', 'properties.dmin', 'properties.rms',
       'properties.gap', 'estado', 'Longitud', 'Latitud', 'Profundidad'],
      dtype='object')

properties.sig : un número que describe la importancia del evento. Los números más grandes indican un evento más significativo. Este valor se determina en función de una serie de factores, que incluyen: magnitud, MMI máximo, informes de sensación e impacto estimado.

properties.dmin Distancia horizontal desde el epicentro hasta la estación más cercana (en grados). 1 grado son aproximadamente 111,2 kilómetros. En general, cuanto menor sea este número, más confiable es la profundidad calculada del terremoto.

	properties.gap: La brecha azimutal más grande entre estaciones azimutalmente adyacentes (en grados). En general, cuanto más pequeño es este número, más confiable es la posición horizontal calculada del terremoto. Las ubicaciones de terremotos en las que la brecha azimutal supera los 180 grados suelen tener grandes incertidumbres de ubicación y profundidad.

In [14]:
df_sismo = df_sismo.rename(columns={'properties.mag':      'Magnituud',   
                                    'properties.place':    'Place',
                                    'properties.time':     'Fecha',  
                                    'properties.updated':  'Ultimo registro', 
                                    'properties.felt':     'Felt',
                                    'properties.cdi':      'cdi',
                                    'properties.mmi':      'mmi', 
                                    'properties.tsunami':  'Posibilidad tsunami',
                                    'properties.sig':      'Importancia del evento',
                                    'properties.ids':      'Ids',
                                    'properties.nst':      'nst',
                                    'properties.dmin':     'Distancia Horizontal Epicentro',
                                    'properties.rms':      'RMS',
                                    'properties.gap':      'Brecha Azimutal'})
df_sismo.head(1)

,id,Magnituud,Place,Fecha,Ultimo registro,Felt,cdi,mmi,Posibilidad tsunami,Importancia del evento,Ids,nst,Distancia Horizontal Epicentro,RMS,Brecha Azimutal,estado,Longitud,Latitud,Profundidad
0,us6000kcxl,2.5,12 km SE of Lincolnville,2023-05-18 19:30:29.827,2023-05-19 03:27:22.499,2.0,4.1,NaN,0,97,",us6000kcxl,",37.0,0.724,0.45,95.0,Kansas,-96.8691,38.4053,5


In [15]:
df_sismo['estado'].unique()

array(['Kansas', 'CA', 'Texas', nan, 'Colorado', 'Oklahoma', 'New Mexico',
       'Mexico', 'Arizona', 'Montana', 'Idaho', 'Nevada', 'Washington',
       None, 'Tennessee', 'B.C.', 'Wyoming', 'Canada', 'New Hampshire',
       'New York', 'California', 'Utah', 'Ohio', 'Nebraska', 'Oregon',
       'Missouri', 'Virginia', 'Maine', 'Kentucky', 'Arkansas',
       'Minnesota', 'Mississippi', 'North Carolina', 'Quebec',
       'Louisiana', 'South Carolina', 'Georgia', 'Alabama',
       'South Dakota', 'Illinois', 'NV', 'United States', 'Florida',
       'Iowa', 'MT', 'Maryland', 'NM', 'Michigan', 'ID', 'UT', 'OK', 'TN',
       'Massachusetts', 'TX', 'Pennsylvania', 'New Jersey',
       'NV Earthquake', 'KS', 'West Virginia', 'WA', 'WY', 'MO',
       'Delaware', 'Indiana'], dtype=object)

## Normalizacion de la columna estado

In [16]:
df_sismo['estado'] = df_sismo['estado'].replace({'United States': "Sin Dato",
                                                 'Sandywoods Township': "Sin Dato",
                                                 "CA":"California",
                                                 "WA": "Washington",
                                                 "WY": "Wyoming",
                                                 "UT": "Utah",
                                                 "TN": "Tennessee",
                                                 "TX": "Texas",
                                                 "ID": "Idaho",
                                                 "KS": "Kansas",
                                                 "MO": "Misuri",
                                                 "MT": "Montana",
                                                 "NM": "Nuevo Mexico",
                                                 "Mexico": "New Mexico",
                                                 "NV": "Nevada",
                                                 "NV Earthquake":"Nevada",
                                                 "OK": "Oklahoma",
                                                 'MN': "Minnesota",
                                                 'CO': 'Colorado',
                                                 'AZ': 'Arizona',
                                                 'OR': 'Oregon',
                                                 'Son.': 'Sonora',
                                                 'CA Earthquake': 'California',
                                                 'California Earthquake': "California",
                                                 'NE':"Nebraska"})

In [17]:
df_sismo['estado'].unique()

array(['Kansas', 'California', 'Texas', nan, 'Colorado', 'Oklahoma',
       'New Mexico', 'Arizona', 'Montana', 'Idaho', 'Nevada',
       'Washington', None, 'Tennessee', 'B.C.', 'Wyoming', 'Canada',
       'New Hampshire', 'New York', 'Utah', 'Ohio', 'Nebraska', 'Oregon',
       'Missouri', 'Virginia', 'Maine', 'Kentucky', 'Arkansas',
       'Minnesota', 'Mississippi', 'North Carolina', 'Quebec',
       'Louisiana', 'South Carolina', 'Georgia', 'Alabama',
       'South Dakota', 'Illinois', 'Sin Dato', 'Florida', 'Iowa',
       'Maryland', 'Nuevo Mexico', 'Michigan', 'Massachusetts',
       'Pennsylvania', 'New Jersey', 'West Virginia', 'Misuri',
       'Delaware', 'Indiana'], dtype=object)

In [18]:
df_sismo.isnull().sum()

id                                    0
Magnituud                             0
Place                                47
Fecha                                 0
Ultimo registro                       0
Felt                              10124
cdi                               10124
mmi                               17240
Posibilidad tsunami                   0
Importancia del evento                0
Ids                                   0
nst                                5551
Distancia Horizontal Epicentro      891
RMS                                   2
Brecha Azimutal                      18
estado                              385
Longitud                              0
Latitud                               0
Profundidad                           0
dtype: int64

In [19]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              20000 non-null  object        
 1   Magnituud                       20000 non-null  float64       
 2   Place                           19953 non-null  object        
 3   Fecha                           20000 non-null  datetime64[ns]
 4   Ultimo registro                 20000 non-null  datetime64[ns]
 5   Felt                            9876 non-null   float64       
 6   cdi                             9876 non-null   float64       
 7   mmi                             2760 non-null   float64       
 8   Posibilidad tsunami             20000 non-null  int64         
 9   Importancia del evento          20000 non-null  int64         
 10  Ids                             20000 non-null  object        
 11  ns

Cambio el tipo de dato de Latitud y Longitud a float para poder utilizarlo en el mapa.

In [20]:
df_sismo["Longitud"] = df_sismo["Longitud"].astype(float) 
df_sismo["Latitud"] = df_sismo["Latitud"].astype(float) 
df_sismo["Profundidad"] = df_sismo["Profundidad"].astype(float)

DATOS UTILIZADOS

Alert
  - Tipo de datos: Cadena
  - Valores típicos: “verde”, “amarillo”, “naranja”, “rojo”.
  - Descripción: El nivel de alerta de la escala de impacto de terremotos de PAGER.  


**cdi**
  - Tipo de datos: decimal
  - Descripción: La máxima intensidad reportada para el evento. Calculado por DYFI. S 
i bien generalmente se informa como un número romano, para los fines de esta API, se espera que la intensidad  
sea el equivalente decimal del número romano. Obtenga más información sobre la magnitud frente a la intensidad. 
 

Depth 
- Tipo de datos: decimal 
- Profundidad del evento en kilómetros.

DepthError
  - Tipo de datos: decimal
  - Valores típicos: [0, 100]- Descripción: Incertidumbre de la profundidad reportada del evento en kilómetros.
  - Información adicional: El error de profundidad, en km, definido como la mayor proyección de los tres errores principales en una línea vertical. 


Dmin 
  - Tipo de datos: decimal
  - Descripción :Distancia horizontal desde el epicentro hasta la estación más cercana (en grados). 1 grado son aproximadamente 111,2 kilómetros.  
En general, cuanto menor sea este número, más confiable es la profundidad calculada del terremoto.

Gap
  - Tipo de datos: decimal
  - Descripción: La mayor brecha azimutal entre estaciones azimutalmente adyacentes (en grados). En general, cuanto más pequeño es este número, más confiable es la posición horizontal calculada del terremoto. Las ubicaciones de terremotos en las que la brecha azimutal supera los 180 grados suelen tener grandes incertidumbres de ubicación y profundidad.

HorizontalError
  - Tipo de datos: decimal
  - Descripción: Incertidumbre de la ubicación reportada del evento en kilómetros.
  - Información adicional: El error de ubicación horizontal, en km, definido como la longitud de la mayor proyección de los tres errores principales en un plano horizontal.  
Los errores principales son los ejes principales del elipsoide de error y son mutuamente perpendiculares.  
Las incertidumbres horizontales y verticales en la ubicación de un evento varían desde aproximadamente 100 m en horizontal y 300 metros en vertical para los eventos mejor ubicados, aquellos en medio de redes de sismógrafos densamente espaciadas, hasta decenas de kilómetros para eventos globales en muchas partes del mundo.  
Informamos un valor "desconocido" si la red sísmica contribuyente no proporciona estimaciones de incertidumbre. 

Latitud
  - Tipo de datos: decimal
  - Descripción :Grados decimales de latitud. Valores negativos para latitudes del sur.
  - Información adicional: Un terremoto comienza a romperse en un hipocentro que está definido por una posición en la superficie de la tierra (epicentro) y una profundidad por debajo de este punto (profundidad focal). Proporcionamos las coordenadas del epicentro en unidades de latitud y longitud. La latitud es el número de grados al norte (N) o al sur (S) del ecuador y varía de 0 en el ecuador a 90 en los polos.


LocationSource 
  - Tipo de datos: Cadena
  - Descripción: la red que originalmente creó la ubicación informada de este evento.

Longitud
  - Tipo de datos: decimal
  - Descripción :Grados decimales de longitud. Valores negativos para longitudes occidentales.
  - Información adicional: La longitud varía de 0 en Greenwich a 180 y el E o W muestra la dirección desde Greenwich. Las coordenadas se dan en el marco de referencia WGS84. La incertidumbre de la posición del hipocentro varía desde aproximadamente 100 m en horizontal y 300 metros en vertical para los eventos mejor ubicados, aquellos en medio de redes de sismógrafos densamente espaciadas, hasta decenas de kilómetros para eventos globales en muchas partes del mundo. 


mag 
 
- Información adicional: La magnitud reportada es la que el Servicio Geológico de los EE. UU. considera oficial para este terremoto, y fue la mejor estimación disponible del tamaño del terremoto, en el momento en que se creó esta página.

Mag Error
  - Tipo de datos: decimal
  - Descripción: Incertidumbre de la magnitud reportada del evento. El error estándar estimado de la magnitud. La incertidumbre corresponde al tipo de magnitud específica que se informa y no tiene en cuenta las variaciones de magnitud y los sesgos entre diferentes escalas de magnitud. Informamos un valor "desconocido" si la red sísmica contribuyente no proporciona estimaciones de incertidumbre.  
  

magNST
  - Tipo de datos: entero
  - - Descripción: El número total de estaciones sísmicas utilizadas para calcular la magnitud de este terremoto.

mmi
  - Tipo de datos: decimal
  - Descripción: La intensidad instrumental máxima estimada para el evento. Calculado por ShakeMap. Si bien generalmente se informa como un número romano, para los fines de esta API, se espera que la intensidad sea el equivalente decimal del número romano. Obtenga más información sobre la magnitud frente a la intensidad.


Nph
  - Número de fases utilizadas: Cadena
  - Descripción: número de observaciones de tiempo de llegada P y S utilizadas para calcular la ubicación del hipocentro. Un mayor número de observaciones de tiempo de llegada generalmente da como resultado mejores ubicaciones de terremotos.

Nst
  - Tipo de datos: entero
  - Descripción: El número total de estaciones sísmicas utilizadas para determinar la ubicación del terremoto.
  - Información Adicional: Número de estaciones sísmicas que reportaron tiempos de llegada P y S para este sismo. Este número: puede ser mayor que el Número de fases utilizadas si se rechazan los tiempos de llegada porque la distancia a una estación sísmica excede la distancia máxima permitida o porque la observación del tiempo de llegada es inconsistente con la solución.

Place
  - Tipo de datos: Cadena
  - - Descripción: Descripción textual de la región geográfica nombrada cercana al evento. Puede ser el nombre de una ciudad o el nombre de una región

Rms 
  - Tipo de datos: decimal
  - Descripción: El residual de tiempo de viaje de la raíz cuadrada media (RMS), en segundos, usando todos los pesos. Este parámetro proporciona una medida del ajuste de los tiempos de llegada observados a los tiempos de llegada previstos para esta ubicación. Los números más pequeños reflejan un mejor ajuste de los datos. El valor depende de la precisión del modelo de velocidad utilizado para calcular la ubicación del terremoto, los pesos de calidad asignados a los datos de tiempo de llegada y el procedimiento utilizado para localizar el terremoto. 

Sig (para discutir)
  - Tipo de datos: entero
  - Descripción: Un número que describe cuán significativo es el evento. Los números más grandes indican un evento más significativo. Este valor se determina en función de una serie de factores, que incluyen: magnitud, MMI máximo, informes de sensación e impacto estimado.

time
  - Tipo de datos: entero largo
  - Información: Indicamos la fecha y hora en que el sismo inicia la ruptura, lo que se conoce como hora "origen". Tenga en cuenta que los grandes terremotos pueden continuar rompiéndose durante muchas decenas de segundos. Proporcionamos la hora en UTC (Tiempo Universal Coordinado). Los sismólogos usan UTC para evitar la confusión causada por las zonas horarias locales y el horario de verano. En las páginas de eventos individuales, también se proporcionan los tiempos en el epicentro y su hora local en función de la hora configurada en su computadora.


tsunami
  - Tipo de datos: entero
  - Descripción: esta bandera se establece en "1" para grandes eventos en regiones oceánicas y "0" en caso contrario. La existencia o el valor de esta bandera no indica si realmente existió o existirá un tsunami.


Tipy(para discutir)
  - Tipo de datos: Cadena
  - Descripción :Tipo de evento sísmico.

Tz
  - Tipo de datos: entero
  - Descripción: Desplazamiento de zona horaria desde UTC en minutos en el epicentro del evento.

Updated
  - Tipo de datos: entero largo
  - Descripción: hora en que se actualizó el evento por última vez. Los tiempos se informan en milisegundos desde la época

Status
  - Tipo de datos: Cadena
  - Descripción: Indica si el evento ha sido revisado por un humano.

idss
  - Tipo de datos: Cadena
  - Descripción: una lista separada por comas de ID de eventos que están asociados a un evento.


In [21]:
df_sismo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              20000 non-null  object        
 1   Magnituud                       20000 non-null  float64       
 2   Place                           19953 non-null  object        
 3   Fecha                           20000 non-null  datetime64[ns]
 4   Ultimo registro                 20000 non-null  datetime64[ns]
 5   Felt                            9876 non-null   float64       
 6   cdi                             9876 non-null   float64       
 7   mmi                             2760 non-null   float64       
 8   Posibilidad tsunami             20000 non-null  int64         
 9   Importancia del evento          20000 non-null  int64         
 10  Ids                             20000 non-null  object        
 11  ns

In [22]:
df_sismo.isnull().sum()

id                                    0
Magnituud                             0
Place                                47
Fecha                                 0
Ultimo registro                       0
Felt                              10124
cdi                               10124
mmi                               17240
Posibilidad tsunami                   0
Importancia del evento                0
Ids                                   0
nst                                5551
Distancia Horizontal Epicentro      891
RMS                                   2
Brecha Azimutal                      18
estado                              385
Longitud                              0
Latitud                               0
Profundidad                           0
dtype: int64

In [23]:
#folder_path = "../DASHBOARD/CSV_TRANSFORMADOS"
#file_name = "Datos_USA.csv"
#file_path = os.path.join(folder_path, file_name)
#
#df_sismo.to_csv(file_path, index=False)